In [1]:
import pickle

import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
with open("data/sample_000000.pkl", "rb") as f:
    sample = pickle.load(f)

In [3]:
points = sample['points']
print("Points shape:", points.shape)
print("Frame ID:", sample['frame_id'])
print("Sample keys:", sample.keys())
print(sample['points'])

Points shape: (16384, 4)
Frame ID: 000000
Sample keys: dict_keys(['frame_id', 'gt_boxes', 'points', 'flip_x', 'noise_rot', 'noise_scale', 'lidar_aug_matrix', 'use_lead_xyz', 'image_shape'])
[[15.322624   -0.9945517  -1.3005242   0.37      ]
 [16.28796     5.4508047  -0.45010167  0.32      ]
 [11.225035    7.8189     -1.5432059   0.21      ]
 ...
 [10.327182    5.269324   -1.5712075   0.4       ]
 [13.737979   -5.4365177  -1.3347485   0.22      ]
 [12.59426    10.674768    0.26653486  0.27      ]]


In [4]:
CLASS_NAMES = ['Car', 'Pedestrian', 'Cyclist']

CLASS_COLORS = {
    1: (1, 0, 0),    # Car     (red)
    2: (0, 1, 0),    # Pedestrian (green)
    3: (0, 0, 1)     # Cyclist (blue)
}

In [12]:
points = sample['points'][:, :3]

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)

z = points[:, 2]
z_norm = (z - z.min()) / (z.max() - z.min() + 1e-6)
colors = plt.cm.jet(z_norm)[:, :3]
pcd.colors = o3d.utility.Vector3dVector(colors)

gt_boxes = sample['gt_boxes']

def get_gt_oriented_bounding_boxes(gt_boxes):
    boxes = []
    for box in gt_boxes:
        x, y, z, dx, dy, dz, heading, class_idx = box
        class_idx = int(class_idx)
        obb = o3d.geometry.OrientedBoundingBox(
            center=[0, 0, 0],
            R=o3d.geometry.get_rotation_matrix_from_xyz([0, 0, heading]),
            extent=[dx, dy, dz]
        )
        obb.translate([x, y, z])
        obb.color = CLASS_COLORS.get(class_idx, (1, 1, 1))
        boxes.append(obb)
    return boxes

o3d.visualization.draw_geometries([pcd, *get_gt_oriented_bounding_boxes(gt_boxes)])

In [6]:
label_path = "/home/tom/kitti/training/label_2/000000.txt"

labels = []
with open(label_path, "r") as f:
    for line in f.readlines():
        fields = line.strip().split(" ")
        obj = {
            "type": fields[0],                    # Car, Pedestrian, Cyclist, ...
            "truncation": float(fields[1]),
            "occlusion": int(fields[2]),
            "alpha": float(fields[3]),
            "bbox": list(map(float, fields[4:8])),       # [xmin, ymin, xmax, ymax]
            "dimensions": list(map(float, fields[8:11])),# [h, w, l]
            "location": list(map(float, fields[11:14])), # [x, y, z] in camera coords
            "rotation_y": float(fields[14])              # rotation around y-axis
        }
        labels.append(obj)

labels

[{'type': 'Pedestrian',
  'truncation': 0.0,
  'occlusion': 0,
  'alpha': -0.2,
  'bbox': [712.4, 143.0, 810.73, 307.92],
  'dimensions': [1.89, 0.48, 1.2],
  'location': [1.84, 1.47, 8.41],
  'rotation_y': 0.01}]

In [7]:
def inverse_rigid_transform(transform):
    inv = np.zeros_like(transform)
    R = transform[0:3, 0:3]
    t = transform[0:3, 3]
    inv[0:3, 0:3] = R.T
    inv[0:3, 3] = -R.T @ t
    inv[3, 3] = 1.0
    return inv


def transform_points(points, transform):
    points_hom = np.c_[points, np.ones(points.shape[0])]
    transformed = points_hom @ transform.T
    return transformed[:, :3]

def compute_box_3d(label, transform):
    h, w, l = label["dimensions"]
    x, y, z = label["location"]
    ry = label["rotation_y"]
    x_corners = [ l/2,  l/2, -l/2, -l/2,  l/2,  l/2, -l/2, -l/2]
    y_corners = [    0,    0,    0,    0,  -h,  -h,  -h,  -h]
    z_corners = [ w/2, -w/2, -w/2,  w/2,  w/2, -w/2, -w/2,  w/2]
    corners = np.vstack([x_corners, y_corners, z_corners])
    R = np.array([
        [ np.cos(ry), 0, np.sin(ry)],
        [          0, 1,          0],
        [-np.sin(ry), 0, np.cos(ry)]
    ])
    corners_3d = R @ corners + np.array([[x], [y], [z]])
    return transform_points(corners_3d.T, transform)

In [8]:
def parse_calib_file(calib_file_path):
    calib_keys = ['P0', 'P1', 'P2', 'P3', 'R0','Tr_velo_to_cam', 'Tr_imu_to_velo']
    calibration_matrices = dict()
    with open(calib_file_path, "r") as file:
        calib_lines = file.readlines()
        for i, key in enumerate(calib_keys):
            elems = calib_lines[i].split(' ')
            elems = elems[1:]
            calib_matrix = np.array(elems, dtype=np.float32)
            if key == 'R0':
                calib_matrix_shape = (3, 3)
            else:
                calib_matrix_shape = (3, 4)
            calib_matrix = calib_matrix.reshape(calib_matrix_shape)
            calibration_matrices[key] = calib_matrix
    return calibration_matrices

calib_file_path = "/home/tom/kitti/training/calib/000000.txt"
calib = parse_calib_file(calib_file_path)
Tr_velo_to_cam = calib['Tr_velo_to_cam']
Tr_velo_to_cam = np.vstack((Tr_velo_to_cam, [0, 0, 0, 1]))
Tr_cam_to_velo = inverse_rigid_transform(Tr_velo_to_cam)
Tr_cam_to_velo

array([[ 0.00692796, -0.00116298,  0.99997532,  0.33219371],
       [-0.99997222,  0.00274984,  0.00693114, -0.02210627],
       [-0.00275783, -0.99999553, -0.0011439 , -0.06171977],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [9]:
gt_oriented_boxes = []
for label in labels:
    corners_3d_velo = compute_box_3d(label, Tr_cam_to_velo)
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(corners_3d_velo)
    oriented_bbox = pcd.get_oriented_bounding_box()
    oriented_bbox.color = (0, 1, 0)
    gt_oriented_boxes.append(oriented_bbox)
print(gt_oriented_boxes)

[OrientedBoundingBox: center: (8.75412, -1.80232, -0.601412), extent: 1.89, 1.2, 0.48)]


In [11]:
bin_path = "/home/tom/kitti/training/velodyne/000000.bin"
points = np.fromfile(bin_path, dtype=np.float32).reshape(-1, 4)

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points[:, :3])

z = points[:, 2]
z_norm = (z - z.min()) / (z.max() - z.min() + 1e-6)
colors = plt.cm.jet(z_norm)[:, :3]
pcd.colors = o3d.utility.Vector3dVector(colors)

o3d.visualization.draw_geometries([pcd, *gt_oriented_boxes])